# Implement a 4x4 gridworld, find terminal with DP


<u>Group:</u><br>
$383321$<br>
$2373769$ <br>
$2376182$<br>
$0709942R$

In [64]:
from copy import deepcopy
from enum import Enum
import numpy as np
import matplotlib.pyplot as plt

## Gridworld
Gridworld for a simple finite MDP. Only actions with movements in cardinal directions are allowed.<br>
And the gird boundaries and obstacles are inaccessible by the agent.

In [ ]:
class CellType(Enum):
    SPACE = 0
    OBSTACLE = 1
    
class Actions(Enum):
    UP = 0
    DOWN = 1
    LEFT = 2
    RIGHT = 3

class State(object):
    def __init__(self, row, col):
        self.row = row
        self.col = col
        
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col
    
    def __str__(self):
        return str((self.row, self.col))
    
        
class GridWorld(object):
        
    def __init__(self, height=4, width=4):
        self.start = State(height-1, 0)
        self.goal  = State(0, 0)
        self.environment = self._init_environment(height, width, True)
        self.rewards = self._init_rewards(self.environment)
        self.shape = (len(self.environment),len(self.environment[0]))
        self.agent_state = self.start 
        
    # Add agent to environment using 4 as agen't position
    def __str__(self):
        environment = self.environment.copy()
        environment[self.agent_state.row, self.agent_state.col] = 4
        return str(environment)
        
        
    def _init_rewards(self, environment):
        rewards = {}
        for row in range(len(environment)):
            for col in range(len(environment[row,:])):
                if environment[row, col] != CellType.OBSTACLE.value:
                    cardinal_states = self.get_cardinal_states(State(row, col))
                    rewards[row, col] = {}
                    for neighbour in cardinal_states:
                        reward = -1
                        if neighbour == self.goal:
                            reward = 20
                        rewards[row, col][neighbour.row, neighbour.col] = reward
        return rewards
    
                    
    def _init_environment(self, height, width, no_obstacles = False):
        environment = []
        for row in range(height):
            new_col = []
            for col in range(width):         
                new_col.append(CellType.SPACE.value)
            environment.append(new_col)
        return np.flip(np.array(environment),axis=0)
    
    def get_agent_state(self, agent):
        return self.agent_state
    
    def get_cardinal_states(self, state):
        return [State(state.row + 1, state.col), State(state.row - 1, state.col),
                State(state.row, state.col + 1), State(state.row, state.col - 1)]

    
    def get_available_actions(self):
        return [Actions.DOWN, Actions.UP, Actions.LEFT, Actions.RIGHT]
    
    def state_reward_from_action(self, state, action): 
        from_state = state
        if action == Actions.DOWN:
            if state.row == len(self.environment) - 1:
            #Grid boundary, go nowhere, penalise
                new_state = state
                reward =  -1
            else:
                new_state = State(state.row + 1, state.col)
                reward = self.rewards[from_state.row, from_state.col][new_state.row, new_state.col]
        if action == Actions.UP:
            if state.row == 0:
            #Grid boundary, go nowhere, penalise
                new_state = state
                reward =  -1
            else:
                new_state = State(state.row - 1, state.col)
                reward = self.rewards[from_state.row, from_state.col][new_state.row, new_state.col]
        if action == Actions.LEFT:
            if state.col == 0:
            #Grid boundary, go nowhere, penalise
                new_state = state
                reward = -1
            else:
                new_state = State(state.row, state.col - 1)
                reward = self.rewards[from_state.row, from_state.col][new_state.row, new_state.col]
        if action == Actions.RIGHT:
            if state.col == len(self.environment) - 1:
            #Grid boundary, go nowhere, penalise
                new_state = state
                reward = -1
            else:
                new_state = State(state.row, state.col + 1)
                reward = self.rewards[from_state.row, from_state.col][new_state.row, new_state.col]
        return new_state, reward
        

## Two-array policy evalution

$v_0$ is intialized arbitrarily, except the terminal state, must be given value 0.

In [119]:
def two_array_policy_eval(grid_world, gamma= 1, pi= 0.01):
    # Start with a arbitrary (all 0) value function
    v_prev = np.zeros(grid_world.shape)
    loop_count = 0
    while True:
        delta = 0
        loop_count += 1   
        
            
        #new value function
        v_new = np.zeros(grid_world.shape)


        #Loop through all states (Env cells)
        for row in range (grid_world.shape[0]): 
            for col in range(grid_world.shape[1]):
                if State(row, col) == grid_world.goal:
                    continue
                    
                #Reset value 
                v_new[row, col] = 0
                
                #loop over possible actions
                for action in grid_world.get_available_actions():
                
                    #get transitions
                    new_state, reward = grid_world.state_reward_from_action(State(row, col), action)
                    
                    #Update value for this action
                    v_new[row, col] += 0.25 * (reward + gamma * v_prev[new_state.row, new_state.col])

                #get the biggest difference over state space
                delta = max(delta, abs(v_new[row, col] - v_prev[row, col]))
                
        #if true value function
        if(delta < pi):
            break

        v_prev = v_new
        

    return np.array(v_prev), loop_count

grid_world = GridWorld()
v, loop_count = two_array_policy_eval(grid_world)

#expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
#np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

print(v)
#print(expected_v)
print(loop_count)

[[  0.          -8.76205969 -23.77685683 -30.29018519]
 [ -8.76205969 -19.52817688 -28.30732788 -32.83711973]
 [-23.77685683 -28.30732788 -33.11940586 -35.94939323]
 [-30.29018519 -32.83711973 -35.94939323 -37.92962469]]
214


## In-place policy evalution

$v_0$ is intialized arbitrarily, except the terminal state, must be given value 0.

In [117]:
def in_place_policy_eval(grid_world, gamma= 1, pi= 0.01):
    # Start with a arbitrary (all 0) value function
    v = np.zeros(grid_world.shape)
    loop_count = 0
    v_prev = 0
    
    while True:
        delta = 0
        loop_count += 1

        # Loop through all states (Env cells)
        for row in range(grid_world.shape[0]):
            for col in range(grid_world.shape[1]):

                if State(row, col) == grid_world.goal:
                    continue
                    
                v_prev = v[row, col]
                
                v[row, col] = 0
                
                # loop over possible actions
                for action in grid_world.get_available_actions():
                    # get transitions
                    new_state, reward = grid_world.state_reward_from_action(State(row, col), action)
                    
                    # Update value for this action
                    v[row, col] += 0.25 * (reward + gamma * v[new_state.row, new_state.col])

                # get the biggest difference over state space

                delta = max(delta, abs(v[row, col] - v_prev))

        # if true value function
        if (delta < pi):
            break
        
        
    return np.array(v), loop_count

In [118]:
grid_world = GridWorld()
v, loop_count = in_place_policy_eval(grid_world)

#expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
#np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

print(v)
#print(expected_v)
print(loop_count)

[[ 0.          2.90725166 -3.19761903 -4.77070734]
 [ 4.55514318 -1.55070564 -5.35325988 -6.42097448]
 [-1.59309065 -4.32555263 -6.25597734 -6.62806081]
 [-2.59999456 -3.91187323 -4.72517333 -4.73538567]]
19
